# Customizing and Extending Enumerations

In [86]:
from enum import Enum

In [87]:
class Color(Enum):
    red = 1 
    green = 2 
    blue = 3

    def purecolor(self, value):
        return {self: value}

In [88]:
Color.red.purecolor(100),Color.blue.purecolor(255)

({<Color.red: 1>: 100}, {<Color.blue: 3>: 255})

In extend to that, we can override the dunder methods

In [89]:
Color.red

<Color.red: 1>

In [90]:
class Color(Enum):
    red = 1 
    green = 2 
    blue = 3

    def purecolor(self, value):
        return {self: value}
    
    def __repr__(self):
        return f"{self.name.capitalize()} ({self.value})"

In [91]:
Color.red

Red (1)

In [92]:
class Number(Enum):
    ONE = 1 
    TWO = 2 
    THREE = 3 

In [93]:
Number.ONE > Number.TWO

TypeError: '>' not supported between instances of 'Number' and 'Number'

In [13]:
class Number(Enum):
    ONE = 1 
    TWO = 2 
    THREE = 3 

    def __lt__(self, other):
        return isinstance(other, Number) and self.value < other.value

In [14]:
Number.ONE < Number.TWO

True

Even with the `__gt__` method not implemented, Python mirrors the `__lt__`

In [15]:
Number.ONE > Number.TWO

False

In [16]:
Number.ONE == 1

False

In [17]:
class Number(Enum):
    ONE = 1 
    TWO = 2 
    THREE = 3 

    def __lt__(self, other):
        return isinstance(other, Number) and self.value < other.value
    
    def __eq__(self, other):
        if isinstance(other, Number):
            return self is other 
        elif isinstance(other, int):
            return self.value == other 
        else:
            False

In [21]:
Number.ONE == 1 

True

In [22]:
Number.ONE == 1.0

But we have to be careful, since we implemented the `__eq__` method, now the Enumeration elements are not hashable anymor

In [25]:
hash(Number.ONE)

TypeError: unhashable type: 'Number'

In [26]:
Number.ONE < Number.TWO

True

In [27]:
Number.ONE <= Number.TWO

TypeError: '<=' not supported between instances of 'Number' and 'Number'

Insted of manually implement it all, we can import from `functools` the `total_ordering` module, that autocompletes the rest of the operations

In [28]:
from functools import total_ordering

@total_ordering
class Number(Enum):
    ONE = 1 
    TWO = 2 
    THREE = 3 

    def __lt__(self, other):
        return isinstance(other, Number) and self.value < other.value
    
    def __eq__(self, other):
        if isinstance(other, Number):
            return self is other 
        elif isinstance(other, int):
            return self.value == other 
        else:
            False

In [29]:
Number.ONE <= Number.TWO

True

## Another Example

In [109]:
class Phase(Enum):
    READY = 'ready'
    RUNNING = 'running'
    FINISHED = 'finished'

    @staticmethod
    def get_element_index(element):
        all_elements_list = list(Phase)
        try:
            element  = Phase(element)
            element_index =  all_elements_list.index(element)
            return element_index
        except ValueError:
            raise ValueError(f"Item '{element}' not found in enumeration class {all_elements_list}.")

    def __str__(self) -> str:
        return f"{self.value}"
    
    def __eq__(self, other) -> bool:
        if isinstance(other, Phase):
            return self is other
        elif isinstance(other, str):
            return self.value == other.strip().lower() 
        
    def __lt__(self, other):
        self_order_inex = Phase.get_element_index(self)
        other_order_index = Phase.get_element_index(other)
        return self_order_inex < other_order_index


In [106]:
Phase.READY < Phase.RUNNING

True

In [107]:
Phase.READY < 'running'

True

In [112]:
Phase.READY < 'ok'

teste


TypeError: '<' not supported between instances of 'int' and 'NoneType'

By default, all enumerations elemnets are `True`

In [113]:
class State(Enum):
    READY = 1 
    BUSY = 0 

In [115]:
bool(State.READY), bool(State.BUSY)

(True, True)

In [118]:
state = State.BUSY
if state is State.READY:
    print('system ready to process next item')
else:
    print('system is busy')

system is busy


In [117]:
state = State.BUSY
if state:
    print('system ready to process next item')
else:
    print('system is busy')

system ready to process next item


In [119]:
class State(Enum):
    READY = 1 
    BUSY = 0 

    def __bool__(self):
        return bool(self.value)

In [122]:
state = State.BUSY

if state:
    print('system ready to process next item')
else:
    print('system is busy')

system is busy


## Enumeration extensions

First, to remind that extensions are not possible:

In [123]:
class Color(Enum):
    RED = 1
    GREEN = 2 
    BLUE = 3 

In [124]:
class ColorAlpha(Color):
    PINK = 4

TypeError: ColorAlpha: cannot extend enumeration 'Color'

But we do __CAN__ extend the methods, but we cannot use the subclass to extend the members

In [125]:
class ColorBase(Enum):
    def hello(self):
        return f"{self} says hello"

In [126]:
class Color(ColorBase):
    RED = 1 
    GREEN = 2 
    BLUE = 3

In [128]:
Color.RED.hello()

'Color.RED says hello'

In [129]:
Color.BLUE.hello

<bound method ColorBase.hello of <Color.BLUE: 3>>

Since we can extend methods, we can use it to override the dunder methods to actual Enum classes with members:

In [130]:
from functools import total_ordering

In [139]:
@total_ordering
class OrderedEnum(Enum):
    """Create an ordering based on the member values.
    So member values have to support rich comparisons"""

    def __lt__(self, other):
        if isinstance(other, OrderedEnum):
            return self.value < other.value
        else:
            return NotImplemented

In [140]:
class Number(OrderedEnum):
    ONE = 1 
    TWO = 2
    THREE = 3

In [141]:
class Dimensio(OrderedEnum):
    D1 = 1,
    D2 = 1, 1
    D3 = 1, 1, 1

In [142]:
Number.ONE < Number.TWO

True

In [145]:
Dimensio.D2 > Dimensio.D1

True

In [146]:
from http import HTTPStatus
type(HTTPStatus)

enum.EnumMeta

In [148]:
list(HTTPStatus)[:10]

[<HTTPStatus.CONTINUE: 100>,
 <HTTPStatus.SWITCHING_PROTOCOLS: 101>,
 <HTTPStatus.PROCESSING: 102>,
 <HTTPStatus.EARLY_HINTS: 103>,
 <HTTPStatus.OK: 200>,
 <HTTPStatus.CREATED: 201>,
 <HTTPStatus.ACCEPTED: 202>,
 <HTTPStatus.NON_AUTHORITATIVE_INFORMATION: 203>,
 <HTTPStatus.NO_CONTENT: 204>,
 <HTTPStatus.RESET_CONTENT: 205>]

In [149]:
HTTPStatus(400)

<HTTPStatus.BAD_REQUEST: 400>

In [152]:
HTTPStatus.OK, HTTPStatus.OK.name, HTTPStatus.OK.value

(<HTTPStatus.OK: 200>, 'OK', 200)

`HTTPStatus` enumeration, besides name and value, also have the `phrase` property

In [153]:
HTTPStatus.NOT_FOUND.value, HTTPStatus.NOT_FOUND.name, HTTPStatus.NOT_FOUND.phrase

(404, 'NOT_FOUND', 'Not Found')

Lets see how to implement something like that ourselves *without* metaprogramming:

In [154]:
class AppsStatus(Enum):
    OK = (0, 'No problem')
    FAILED = (1, 'Crap!')

In [155]:
AppsStatus.OK 

<AppsStatus.OK: (0, 'No problem')>

In [158]:
AppsStatus.OK.value[0], AppsStatus.OK.value[1]

(0, 'No problem')

In [159]:
class AppsStatus(Enum):
    OK = (0, 'No problem')
    FAILED = (1, 'Crap!')

    @property
    def code(self):
        return self.value[0]
    
    @property
    def phrase(self):
        return self.value[1]

In [161]:
AppsStatus.OK.code, AppsStatus.OK.phrase

(0, 'No problem')

We have a problem here: we cannot lookup the status using the code like we did in the `HTTPStatus(200)`

In [162]:
HTTPStatus(200)

<HTTPStatus.OK: 200>

In [163]:
AppsStatus(0)

ValueError: 0 is not a valid AppsStatus

In [167]:
AppsStatus((0,'No problem'))

<AppsStatus.OK: (0, 'No problem')>

We solve that with the `__new__` method

In [170]:
class AppsStatus(Enum):
    OK = (0, 'No problem')
    FAILED = (1, 'Crap!')

    def __new__(cls, member_value, member_phrase):
        print(f"__new__(cls={cls}, member_value={member_value}, member_phrase={member_phrase})")
        # the new must return a member object, an instance of AppStatus, but how do we do that in the class constructor??
        # delegated back to the object native python class!
        member = object.__new__(cls) 
        member._value_  = member_value
        member.phrase = member_phrase 
        return member  

__new__(cls=<enum 'AppsStatus'>, member_value=0, member_phrase=No problem)
__new__(cls=<enum 'AppsStatus'>, member_value=1, member_phrase=Crap!)


In [173]:
AppsStatus.OK.value, AppsStatus.OK.name, AppsStatus.OK.phrase

(0, 'OK', 'No problem')

In [176]:
AppsStatus(0).phrase

'No problem'

An combined that with subclassing, we can make this a base for other classes:

In [177]:
class TwoValueEnum(Enum):
    def __new__(cls, member_value, member_phrase):
        member = object.__new__(cls)
        member._value_ = member_value
        member.phrase = member_phrase
        return member

In [178]:
class AppsStatus(TwoValueEnum):
    OK = (0, 'No problem')
    FAILED = (1, 'Crap!')

In [180]:
AppsStatus(1).phrase

'Crap!'